In [ ]:
import os
import re
import xml.etree.ElementTree as ET
from sklearn import metrics
import pandas as pd
from sklearn.metrics import confusion_matrix
#import numpy as np
#np.set_printoptions(threshold=np.inf)

#--------------------------------------------------------------------------------------------------------------

word_dict = dict()
train_data = open('word_tag_frequency.txt', 'w',encoding="utf8")
accuracy = open('accuracy.txt', 'w',encoding="utf8")

#--------------------------------------------------------------------------------------------------------------

def update_word_dict(text,x):
    word=re.split('-',x.attrib['c5'])
    for w in word:
        if text in word_dict:
                if w in word_dict[text]:
                    word_dict[text][w] += 1
                else:
                    word_dict[text][w] = 1

        else:
            word_dict[text] = {}
            word_dict[text][w] = 1

#---------------------------------------------Accuracy----------------------------------------------------------

directory1= r"/home/daddy_yankee/Desktop/csn371/csn371_assignment/Train-corpus"
directory2=r"/home/daddy_yankee/Desktop/csn371/csn371_assignment/Test-corpus"

for file in os.listdir(directory1):
    subfile = os.path.join(directory1, file)
    for xml_files in os.listdir(subfile):
        xml_file = os.path.abspath(os.path.join(subfile, xml_files))
        myTree = ET.parse(xml_file)
        myRoot = myTree.getroot()
        for x in myRoot.findall(r'.//w'):
            text = (x.text).strip().lower()
            update_word_dict(text,x)
        for x in myRoot.findall(r'.//c'):
            text = (x.text).strip().lower()
            update_word_dict(text,x)

final = dict()
final_numerator = 0
final_denominator = 0

for w in word_dict:
    train_data.write(w+" ")
    sub_dict = sorted(word_dict[w].items(), key=lambda p: p[1], reverse=True)
    for x in sub_dict:
        train_data.write(x[0]+ "-" + str(x[1]) + " ")
        final[w] = x[0]
        break
    train_data.write('\n')

for file in os.listdir(directory2):
    subfile=os.path.join(directory2,file)
    for xml_files in os.listdir(subfile):
        xml_file=os.path.abspath(os.path.join(subfile,xml_files))
        myTree = ET.parse(xml_file)
        myRoot = myTree.getroot()
        numerator = 0
        denominator = 0
        for x in myRoot.findall(r".//w"):
            if x.text is not None:
                text = (x.text).strip().lower()
                if text in final:
                    w=re.split('-',x.attrib['c5'])
                    if final[text] in w:
                        numerator+=1
                denominator+=1

        for x in myRoot.findall(r".//c"):
            if x.text is not None:
                text = (x.text).strip().lower()
                if text in final:
                    w = re.split('-', x.attrib['c5'])
                    if final[text] in w:
                        numerator += 1
                denominator+=1

        final_numerator+=numerator
        final_denominator+=denominator
        accuracy.write(str(numerator/denominator)+" "+str(numerator)+" "+str(denominator)+'\n')

accuracy.write(str(final_numerator/final_denominator)+" "+str(final_numerator)+" "+str(final_denominator)+'\n')
print("Accuracy = "+str((final_numerator/final_denominator)*100))
print("\n")
train_data.close()
accuracy.close()

#------------------------------------------Confusion Matrix------------------------------------------------------

y_true=[]
y_pred=[]

for file in os.listdir(directory2):
    subfile=os.path.join(directory2,file)
    for xml_files in os.listdir(subfile):
        xml_file=os.path.abspath(os.path.join(subfile,xml_files))
        myTree = ET.parse(xml_file)
        myRoot = myTree.getroot()
        for x in myRoot.findall(r".//w"):
         if x.text is not None:
            text= (x.text).strip().lower()
            w= re.split("-",x.attrib['c5'])
            check= w[0]
            y_true.append(check)
            if text in final:
                if final[text]==check:
                    y_pred.append(check)
                else:
                    y_pred.append(final[text])
            else:
                y_pred.append("NN1")

        for x in myRoot.findall(r".//c"):
         if x.text is not None:
            text= (x.text).strip().lower()
            w= re.split("-",x.attrib['c5'])
            check= w[0]
            y_true.append(check)
            if text in final:
                if final[text]==check:
                    y_pred.append(check)
                else:
                    y_pred.append(final[text])
            else:
                y_pred.append('NN1')

print("Confusion Matrix = ")
print("\n")
print(confusion_matrix(y_true,y_pred))
print("\n")

#---------------------------------------------------------------------------------------------------------------